# LSTM-nätverk

Ett LSTM-nätverk skiljer sig från vanliga ML-metoder, eftersom den har minne: https://en.wikipedia.org/wiki/Long_short-term_memory

Det betyder att den, i stället för en 1D inputvektor, tar en 2D inputmatris med historiska värden för varje element i inputvektorn. En svaghet är att man i förväg måste bestämma hur lång historiken är. Matrisen måste alltid ha samma dimensioner.

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using Theano backend.


## Träna ett LSTM

Den totala inputvektorn är 3D, eftersom varje exempel är 2D. I fallet där allt är en enda tidsserie som man vill förutsäga (säg en aktiekurva och parametrar som kan influera den) så har man stateful=True, nedan. Det betyder att den ska minnas sitt state mellan batches och exempel.

I det generella fallet så har man flera oberoende exempel. T.ex hur bokningsläget ser ut för alla flygavgångar över tid för en viss rutt på ett flygbolag. Då skulle varje avgång vara ett oberoende exempel.

Nedan visar hur man tränar ett LSTM. "neurons" är antal neuroner som ska ingå i nätverket. Fler neuroner ökar risk för överträning och träningstid, men get möjlighet till mer komplexa insikter.

In [2]:
def fit_lstm(X, y, neurons, batch_size=1, nb_epoch=10):
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=False))
    model.add(Dense(1))
    model.compile(loss='mean_absolute_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, batch_size=batch_size, verbose=0, shuffle=False)
        model.reset_states()
    return model

## Testdata

För att se hur väl det funkar, konstruerar vi fyra testfunktioner:
`make_average`- Output är medlet av tidsserien för inputkolumnen.
`make_max`- Output är max av tidsserien för inputkolumnen
`make_lin`- Inputserien ökar linjärt och Output är hur mycket den ökar per tidssteg.

`make_combo`- Input är tre tidsserier och output är Medlet för den första * summan / 10 av den andra + det sista värdet / 10 av den tredje.

Det är viktigt att normalisera data så att input och output ligger i sigmoidfunktionernas aktiva regioner. T.ex genom att använda sklearn MinMaxScaler mellan (-1, 1). I detta fallet är jag noga med att se till att allt redan ligger ungefär mellan -1 och 1, i stället.

In [3]:
import numpy as np
import random

def make_vector(n):
    scale = random.random() * 2 - 1
    return np.array([[random.random() * scale] for _ in range(n)])

def make_vector_n(n, m):
    scale = np.array([random.random() * 2 - 1 for _ in range(m)])
    return np.array([scale * np.array([random.random() for _ in range(m)]) for _ in range(n)])

def make_average(n, rows):
    X = np.array([make_vector(n) for _ in range(rows)])
    return X, X.mean(axis=1)[:,0]

def make_max(n, rows):
    X = np.array([make_vector(n) for _ in range(rows)])
    return X, X.max(axis=1)[:,0]

def make_lin(n, rows):
    def _lin(n):
        step = (random.random() - 0.5) / 10
        state = (random.random() - 0.5) / 10
        ret = []
        for i in range(n):
            ret.append([state])
            state += step
        return ret
    X = np.array([_lin(n) for _ in range(rows)])
    return X, X[:, 1, 0] - X[:, 0, 0]

def make_combo(n, rows):
    X = np.array([make_vector_n(n, 3) for _ in range(rows)])
    return X, X.mean(axis=1)[:,0] * X.sum(axis=1)[:,1] / 10 + X[:, -1, 2] / 10

def train_test(Z, neurons, epochs=10, testnr=10):
    X, y = Z
    model = fit_lstm(X[:-testnr], y[:-testnr], neurons, nb_epoch=epochs)
    p = model.predict(X[-testnr:])
    for a, b in zip(p[:,0], y[-testnr:]):
        print('%.3f %.3f %.3f' % (abs(a - b), a, b))

Nedan tränar jag de olika problem och testar sedan 10 olika inputs och visar 1) det absoluta felet, 2) det predicerade värdet och 3) det riktiga värdet.

In [4]:
train_test(make_average(10, 1000), 5)

0.003 -0.189 -0.186
0.002 -0.415 -0.417
0.005 0.056 0.061
0.000 -0.313 -0.313
0.009 0.563 0.573
0.006 0.419 0.424
0.005 0.099 0.105
0.003 0.322 0.326
0.002 -0.278 -0.276
0.001 0.440 0.441


`Max()` visar sig vara en svår funktion, trots att jag försöker med fler neuroner och längre träning. Förmodligen för att den har en sorts if-sats och diskontinuiteter, som inte är särskilt lämpliga för NN. Kanske blir det bättre om man lägger två LSTM-lager efter varandra?

In [5]:
train_test(make_max(10, 1000), 20, 100)

0.000 0.209 0.209
0.001 0.152 0.152
0.000 -0.194 -0.194
0.000 -0.013 -0.012
0.000 -0.009 -0.009
0.001 0.538 0.539
0.000 0.363 0.363
0.000 -0.030 -0.030
0.000 0.007 0.007
0.000 0.145 0.145


In [9]:
train_test(make_lin(10, 1000), 5)

0.000 -0.003 -0.003
0.001 -0.028 -0.029
0.001 -0.032 -0.033
0.000 -0.018 -0.018
0.002 0.039 0.041
0.002 0.032 0.034
0.000 -0.010 -0.010
0.001 0.000 0.001
0.001 -0.016 -0.016
0.001 0.007 0.008


Även kombinationen av tre tidsserier fungerar förvånansvärt bra!

In [7]:
train_test(make_combo(10, 10000), 5)

0.005 -0.118 -0.113
0.004 -0.035 -0.038
0.001 -0.124 -0.123
0.011 -0.176 -0.186
0.004 -0.086 -0.090
0.003 0.116 0.113
0.002 -0.040 -0.038
0.004 -0.037 -0.041
0.004 0.053 0.049
0.000 0.006 0.007


In [8]:
train_test(make_combo(10, 10000), 20, 10)

0.001 0.020 0.019
0.002 0.178 0.180
0.000 -0.181 -0.181
0.001 -0.056 -0.057
0.000 0.037 0.038
0.001 -0.019 -0.020
0.002 -0.059 -0.057
0.005 0.224 0.220
0.009 0.235 0.226
0.001 -0.084 -0.085


## Sammanfattning

För att göra regression eller förutsäga kategorier på tidsseriedata så är LSTM lovande. LSTM är bättre än vanliga rekursiva neurala nätverk (RNN) på att använda tidiga värden i tidsserien, men den har dock problem om det kan vara viktigt att titta långt tillbaka i tiden på riktigt långa trender. 10 tidssteg, som i dessa exempel, går bra. 100 går också bra. 1000 kan vara svårt om värden "tidigt" i serien är viktiga, tror jag.

LSTM fungerar extra bra i kombination med att man först tagit bort trender och säsongsvariationer ur tidsserien med t.ex Prophet och gör regression med en modifierad tidsserie, där man sedan åter lägger på trend och säsongsvariation på slutresultatet.

Även om det i teorin går att köra bara LSTM på inputs till en tidsserie och få ett resultat av det, så är jag (utan experimentella belägg) skeptisk till att den klarar allt som t.ex ett boostat random forest klarar av. Därför tror jag på att först träna ett LSTM och sedan lägga in output från LSTM tillsammans med data som man aggregerat själv i ett vanligt 1D regressionsproblem.

Jag har inte någon dropout påslagen på LSTM eller det sista Dense-lagret. Om man gör det så hjälper det att motverka överträning.

## Alternativ
Det finns en liknande, nyare (2014), metod som heter Gated Recurrent Unit (GRU) - https://keras.io/layers/recurrent/#gru som man kan använda som drop in replacement i exemplen ovan. Importera GRU i stället för LSTM och gör .add på ett sådant lager i stället. GRU är nyare, men oklart om de är bättre.